<a href="https://colab.research.google.com/github/ShionMatsuoka/For_ExSeed/blob/main/%E9%96%BE%E5%80%A4%E3%81%AE%E6%9C%80%E9%81%A9%E5%8C%96_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gc
import lightgbm as lgb
from sklearn.ensemble import VotingRegressor
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import spacy
import string
import random
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier,BaggingClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn.metrics import cohen_kappa_score
from lightgbm import log_evaluation, early_stopping
from sklearn.linear_model import SGDClassifier
import polars as pl
import joblib
from sklearn.linear_model import LinearRegression

In [ ]:
data=pd.read_csv('/content/valid_df_fold_0_v1.csv')
for i in range(4):
  read=pd.read_csv(f'/content/valid_df_fold_{i+1}_v1.csv')
  data=pd.concat([data,read])




In [ ]:
print(data)

     essay_id                                          full_text  score  \
0     001bdc0  We all heard about Venus, the planet without a...      4   
1     0033037  The posibilty of a face reconizing computer wo...      2   
2     0066c7c  My position on driveless cars are bad because ...      2   
3     0072128  Imagine a world without garages, gastations, o...      4   
4     0079f2a  In the text the author uses facts that people ...      2   
...       ...                                                ...    ...   
3456  ffc9095  Venus, an extraordinary planet because of many...      3   
3457  ffcb264  Using technology is a good way to help other i...      2   
3458  ffd378d  the story " The Challenge of Exploing Venus " ...      2   
3459  ffddf1f  Technology has changed a lot of ways that we l...      4   
3460  fff016d  If you don't like sitting around all day than ...      2   

      label  fold      pred  
0       3.0   0.0  3.486328  
1       1.0   0.0  2.120117  
2       1

In [ ]:
from sklearn.metrics import cohen_kappa_score
def quadratic_weighted_kappa(y_true, y_pred):
    y_true = y_true + a
    y_pred = (y_pred + a).clip(1, 6).round()
    qwk = cohen_kappa_score(y_true, y_pred, weights="quadratic")
    return 'QWK', qwk, True
def qwk_obj(y_true, y_pred):
    labels = y_true + a
    preds = y_pred + a
    preds = preds.clip(1, 6)
    f = 1/2*np.sum((preds-labels)**2)
    g = 1/2*np.sum((preds-a)**2+b)
    df = preds - labels
    dg = preds - a
    grad = (df/g - f*dg/g**2)*len(labels)
    hess = np.ones(len(labels))
    return grad, hess

a = 2.998
b = 1.092
callbacks = [log_evaluation(period=25), early_stopping(stopping_rounds=75,first_metric_only=True)]

In [ ]:
for fold in range(len(data['fold'].unique())):
  train = data[data['fold'] != fold]
  valid = data[data['fold'] == fold]
  X_train_fold, X_test_fold = train['pred'].values, valid['pred'].values #.reshape(-1, 1)
  y_train_fold, y_test_fold = train['score'], valid['score']




In [ ]:
print(data)

     essay_id                                          full_text  score  \
0     001bdc0  We all heard about Venus, the planet without a...      4   
1     0033037  The posibilty of a face reconizing computer wo...      2   
2     0066c7c  My position on driveless cars are bad because ...      2   
3     0072128  Imagine a world without garages, gastations, o...      4   
4     0079f2a  In the text the author uses facts that people ...      2   
...       ...                                                ...    ...   
3456  ffc9095  Venus, an extraordinary planet because of many...      3   
3457  ffcb264  Using technology is a good way to help other i...      2   
3458  ffd378d  the story " The Challenge of Exploing Venus " ...      2   
3459  ffddf1f  Technology has changed a lot of ways that we l...      4   
3460  fff016d  If you don't like sitting around all day than ...      2   

      label  fold      pred  
0       3.0   0.0  3.486328  
1       1.0   0.0  2.120117  
2       1

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split

# QWKカスタム評価関数の定義
def qwk_metric(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.clip(np.round(y_pred), 0, 5).astype(int)  # 0から5の範囲にクリップし、四捨五入して整数に変換
    qwk = cohen_kappa_score(y_true, y_pred, weights="quadratic")
    return qwk

# 最適な閾値を決定する関数
def find_best_thresholds(y_true, y_pred):
    best_qwk = -np.inf
    best_thresholds = [0.5, 1.5, 2.5, 3.5, 4.5]  # 初期値

    for t1 in np.arange(0.5, 1.5, 0.1):
        for t2 in np.arange(1.5, 2.5, 0.1):
            for t3 in np.arange(2.5, 3.5, 0.1):
                for t4 in np.arange(3.5, 4.5, 0.1):
                    for t5 in np.arange(4.5, 5.5, 0.1):
                        thresholds = [t1, t2, t3, t4, t5]
                        y_pred_classes = np.digitize(y_pred, thresholds, right=True)
                        qwk = qwk_metric(y_true, y_pred_classes)
                        if qwk > best_qwk:
                            best_qwk = qwk
                            best_thresholds = thresholds

    return best_thresholds



# 各foldの結果を保存するリストを作成
qwk_scores = []
thresholds_list = []  # 閾値を保存するリスト

# クロスバリデーションを実行
for fold in range(len(data['fold'].unique())):
    # トレーニングデータと検証データを分割
    train = data[data['fold'] != fold]
    valid = data[data['fold'] == fold]

    y_train_fold, y_test_fold = train['label'], valid['label']
    y_pred_fold = valid['pred']

    # 最適な閾値を見つける
    best_thresholds = find_best_thresholds(y_test_fold, y_pred_fold)
    thresholds_list.append(best_thresholds)

    # 最適な閾値を使って予測値を分類
    y_pred_classes = np.digitize(y_pred_fold, best_thresholds, right=True)

    # QWKを計算
    qwk = qwk_metric(y_test_fold, y_pred_classes)
    qwk_scores.append(qwk)

    # 結果を表示
    print(f"Fold {fold}: QWK = {qwk}, Thresholds = {best_thresholds}")

# 全体の平均QWKを表示
print(f"Average QWK: {np.mean(qwk_scores)}")
print(f"Best Thresholds: {thresholds_list}")

Fold 0: QWK = 0.8298384555346395, Thresholds = [1.4, 2.400000000000001, 3.400000000000001, 4.300000000000001, 5.299999999999997]
Fold 1: QWK = 0.8422129859750804, Thresholds = [1.4, 2.400000000000001, 3.3000000000000007, 4.300000000000001, 5.099999999999998]
Fold 2: QWK = 0.8136805118337954, Thresholds = [1.4, 2.400000000000001, 3.3000000000000007, 4.200000000000001, 5.1999999999999975]
Fold 3: QWK = 0.837739128999028, Thresholds = [1.4, 2.400000000000001, 3.400000000000001, 4.200000000000001, 5.1999999999999975]
Fold 4: QWK = 0.8313025691609749, Thresholds = [1.4, 2.400000000000001, 3.400000000000001, 4.4, 5.299999999999997]
Average QWK: 0.8309547303007037
Best Thresholds: [[1.4, 2.400000000000001, 3.400000000000001, 4.300000000000001, 5.299999999999997], [1.4, 2.400000000000001, 3.3000000000000007, 4.300000000000001, 5.099999999999998], [1.4, 2.400000000000001, 3.3000000000000007, 4.200000000000001, 5.1999999999999975], [1.4, 2.400000000000001, 3.400000000000001, 4.200000000000001, 5